# Automated ML
TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
import pkg_resources
import xgboost


from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig


# Overview

TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')


quick-starts-ws-243057
aml-quickstarts-243057
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


In [3]:
experiment_name = 'AutoML_Heart-Failure-experiment'

experiment = Experiment(ws, experiment_name)
print(experiment)


Experiment(Name: AutoML_Heart-Failure-experiment,
Workspace: quick-starts-ws-243057)


In [4]:
amlcompute_cluster_name = "aml-compute1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_DS3_v2',
        min_nodes=1,
        max_nodes=4
    )
    compute_target = ComputeTarget.create(
        ws,
        amlcompute_cluster_name,
        compute_config
    )

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
found = False
key = "Heart-Failure"
description_text = "Heart Failure data set"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    # Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)

df = dataset.to_pandas_dataframe()


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_settings = {
    "n_cross_validations": 2,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4,
    "experiment_timeout_minutes": 20,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(
    compute_target=compute_target,
    task='classification',
    training_data=dataset,
    label_column_name='DEATH_EVENT',
    featurization='auto',
    debug_log="automl_errors.log",
    enable_onnx_compatible_models=False,
    **automl_settings
)


In [7]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)
automl_run.wait_for_completion()


Submitting remote run.
No run_configuration provided, running on aml-compute1 with default configuration
Running on remote compute: aml-compute1


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML_Heart-Failure-experiment,AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

{'runId': 'AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65',
 'target': 'aml-compute1',
 'status': 'Completed',
 'startTimeUtc': '2023-10-21T08:24:04.384815Z',
 'endTimeUtc': '2023-10-21T08:31:31.588441Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'aml-compute1',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML_Heart-Failure-experiment","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-243057","workspace_name":"quick-starts-ws-243057","region":"southcentralus","compute_targe

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(automl_run).show()
for children_run in automl_run.get_children():
    print('------')
    print(children_run)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_38,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_39,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_37,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_36,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_35,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_34,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [36]:
best_run = automl_run.get_best_child()
print(best_run)

brm = best_run.get_metrics()

for metric_name, metric in zip(brm.keys(), brm.values()):
    print(f"{metric_name}: {metric}")


Run(Experiment: AutoML_Heart-Failure-experiment,
Id: AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_38,
Type: azureml.scriptrun,
Status: Completed)
f1_score_micro: 0.8662416107382549
weighted_accuracy: 0.9023216380073948
matthews_correlation: 0.6868352689349047
f1_score_weighted: 0.8613073585402803
precision_score_micro: 0.866241610738255
recall_score_weighted: 0.866241610738255
AUC_weighted: 0.8963772574544879
norm_macro_recall: 0.6395017455054488
average_precision_score_weighted: 0.8997707435335137
f1_score_macro: 0.8364553803205838
precision_score_weighted: 0.8686737497093369
recall_score_macro: 0.8197508727527244
log_loss: 0.40493050978650286
average_precision_score_macro: 0.8754220537607091
recall_score_micro: 0.866241610738255
AUC_micro: 0.9074632323869295
precision_score_macro: 0.8694440231780757
average_precision_score_micro: 0.9056011186894737
AUC_macro: 0.896377257454488
accuracy: 0.866241610738255
balanced_accuracy: 0.8197508727527244
confusion_matrix: aml://artifactId/Experime

In [37]:
print(best_run.properties)


{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'AutoML_Heart-Failure-experiment\',\'compute_target\':\'aml-compute1\',\'subscription_id\':\'61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_6d8521ce-219d-40b5-a0b8-88593c3c2c65_38","experiment_name":"AutoML_Heart-Failure-experiment","workspace_name":"quick-starts-ws-243057","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group_name":"aml-quickstarts-243057"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '38', '_aml_system_scenario_identification': 'Remote.Child

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [44]:
model_name = best_run.properties["model_name"]
script_file_name = "inference/score.py"
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")

description = "Best Model"
tags = None
model = automl_run.register_model(
    model_name=model_name,
    description=description,
    tags=tags
)


In [55]:
# Define your inference configuration (entry script, dependencies, etc.)
inference_config = InferenceConfig(entry_script=script_file_name)

# Configure the ACI deployment
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    description="Endpoint for Heart Failure from AutoML Classification",
    enable_app_insights=True,  # Enable Application Insights
    auth_enabled=True,  # Enable token-based authentication
)

# Deploy the model to ACI
aci_service_name = model_name.lower()
print(aci_service_name)

# Deploy the service
aci_service = Model.deploy(ws, aci_service_name, [
                           model], inference_config, aciconfig)

# Wait for the deployment to complete
aci_service.wait_for_deployment(True)
print(aci_service.state)


automl6d8521ce238
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-10-21 09:51:45+00:00 Creating Container Registry if not exists.
2023-10-21 09:51:46+00:00 Use the existing image.
2023-10-21 09:51:47+00:00 Submitting deployment to compute.
2023-10-21 09:51:52+00:00 Checking the status of deployment automl6d8521ce238..
2023-10-21 09:53:45+00:00 Checking the status of inference endpoint automl6d8521ce238.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [56]:
%run endpoint.py


{"result": [1, 1]}
++++++++++++++++++++++++++++++
Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


TODO: In the cell below, print the logs of the web service and delete the service

In [59]:
logs = aci_service.get_logs()
print(logs)


2023-10-21T09:53:27,614224700+00:00 - rsyslog/run 
2023-10-21T09:53:27,627236900+00:00 - gunicorn/run 
2023-10-21T09:53:27,629731600+00:00 | gunicorn/run | 
2023-10-21T09:53:27,632046300+00:00 | gunicorn/run | ###############################################
2023-10-21T09:53:27,637501400+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-10-21T09:53:27,639536400+00:00 | gunicorn/run | ###############################################
2023-10-21T09:53:27,646177200+00:00 | gunicorn/run | 
2023-10-21T09:53:27,656760300+00:00 | gunicorn/run | 
2023-10-21T09:53:27,679027900+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2023-10-21T09:53:27,680350100+00:00 | gunicorn/run | 
2023-10-21T09:53:27,682017700+00:00 | gunicorn/run | 
2023-10-21T09:53:27,688708800+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
20

In [60]:
aci_service.delete()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.